In [1]:
import pandas as pd
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

### O datech EUROSTATU
Pro spravnou interpretaci a manipulaci s daty o kriminalite z Eurostatu je potreba si nejprve nastudovat potrebnou [dokumentaci](https://ec.europa.eu/eurostat/cache/metadata/en/crim_sims.htm), souhrnne sdeleni vcetne metodiky zpracovani dat popsano v [README.md](/README.rd)

Import a aktivace vlastni tridy, automaticke cisteni, tranformace a statisticky prepocet dat. 

In [2]:
from eurostatlib.crimetable import EurostatCrimeTable

crime_table = EurostatCrimeTable()

geo_df = pd.read_csv(f'../data/geo.csv')
iccs_df = pd.read_csv(f'../data/iccs.csv')

crime_table.load_data(f'../data/estat_crim_off_cat.tsv', geo_df, iccs_df)
crime_table.create_summary_df_1all()
df = crime_table.country_crime_info_11

V pripade, ze bychom chteli ze summarizacni tabulky porovnat data se zakladnimi hodnotami jeste neprepocitaneho df, je mozno zavolat crime_table.filter_data('<nazev_zeme>', '<krimi_cin>') a poté pracovat s vyfiltovanými daty pod crime_table.filtered_data (podoruceno preulozit do promenne).

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 861 entries, 0 to 860
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   country                    861 non-null    object 
 1   crime                      861 non-null    object 
 2   crime_category             861 non-null    object 
 3   count_years                861 non-null    int64  
 4   count_fill_values          861 non-null    int64  
 5   first_fill_year            759 non-null    object 
 6   last_fill_year             759 non-null    object 
 7   mean_value                 759 non-null    float64
 8   median_value               759 non-null    float64
 9   max_value                  759 non-null    float64
 10  max_value_year             759 non-null    object 
 11  min_value                  774 non-null    float64
 12  min_value_year             774 non-null    object 
 13  standard_deviation         759 non-null    float64

V summarizacni tabulce jsou vypocitane a sebirane nejpodstatnejsi udaje, se kterymi lze dale pracovat a dle podminek si data filtrovat. Nejvhodnejsi je pouzit je pro monitoring podezrelych hodnot, tyto data neslouzi primo k vizualizacim. V ramci interaktivniho jupyter notebooku ci dash/plotly app se automatizovane generuje dle vygenerovanych hodnot zhodnocujici text. Nize pro priklad ukazano, jak lze s daty pracovat a co z nich lze vycist. 

In [4]:
df.head(2)

,country,crime,crime_category,count_years,count_fill_values,first_fill_year,last_fill_year,mean_value,median_value,max_value,...,min_value_year,standard_deviation,quality_range_fill_data,quality_range_unfill_data,missing_values_info,trend,relative_trend_strength,min_range_year,max_range_year,count_outliers
0,Albania,Acts against computer systems,hidden,15,7,2016,2022,3.89,2.930,7.14,...,2018,1.77,7,0,The time series has no missing values within t...,increasing,0.68,2008.0,2022.0,0
1,Albania,Attempted intentional homicide,visible,15,12,2008,2022,5.05,4.525,7.63,...,2019,1.67,15,3,The time series has 3 missing value(s) within ...,to missing value(s),NaN,2008.0,2022.0,0


In [5]:
#nula v count_years znamena, ze pro dany kriminalni cin neexistuje zaznam, respektive ze pro nej neexistuje casove období
no_record_crime = df[df['count_years'] == 0] 

#zjistime, kolik zemi nejvice figuruji v nezverejnovani dat o krimi cinech, coz je napr. Bosnia and Herzegovina,England and Wales  
country_no_record_crime = no_record_crime[['country', 'crime']].value_counts().reset_index()[['country', 'crime']]
country_no_record_crime['crime'].value_counts() # seznam kriminalnich cinnu, ktere nejsou zaznamenavany

crime
Acts against computer systems                             13
Participation in an organized criminal group              13
Child pornography                                          9
Sexual exploitation                                        9
Bribery                                                    8
Money laundering                                           7
Corruption                                                 5
Fraud                                                      5
Kidnapping                                                 4
Attempted intentional homicide                             3
Burglary of private residential premises                   2
Sexual violence                                            2
Burglary                                                   2
Rape                                                       2
Sexual assault                                             1
Serious assault                                            1
Unlawful acts invo

Jako nekvalitni data bychom mohli definovat ta, kterym behem zaznamenavaneho obdobi bud chybi vyplnene hodnoty, nebo jsou jejich hodnoty prilis kratke na to, aby se z nich mohla nejak odvozovat trendovost, respektive potvrdit trendovost. V zakladnim dataframu vsak najdeme vypocitane trendy jiz od vyplnenych hodnot 2 vcetne.

In [6]:
# chceme, aby chybela alespon jedna hodnota, nebo byla casova rada kratsi 4let vcetne
no_quality_data = df[(df['quality_range_unfill_data'] != 0) | (df['count_fill_values'] <= 4)]
no_quality_data.head()

,country,crime,crime_category,count_years,count_fill_values,first_fill_year,last_fill_year,mean_value,median_value,max_value,...,min_value_year,standard_deviation,quality_range_fill_data,quality_range_unfill_data,missing_values_info,trend,relative_trend_strength,min_range_year,max_range_year,count_outliers
1,Albania,Attempted intentional homicide,visible,15,12,2008,2022,5.05,4.525,7.63,...,2019,1.67,15,3,The time series has 3 missing value(s) within ...,to missing value(s),NaN,2008.0,2022.0,0
3,Albania,Burglary,visible,15,8,2008,2022,11.34,8.670,26.36,...,2008,8.19,15,7,The time series has 7 missing value(s) within ...,to missing value(s),NaN,2008.0,2022.0,0
4,Albania,Burglary of private residential premises,visible,15,12,2008,2022,35.79,37.185,57.21,...,2022,11.19,15,3,The time series has 3 missing value(s) within ...,to missing value(s),NaN,2008.0,2022.0,0
8,Albania,Intentional homicide,visible,15,12,2008,2022,2.38,2.095,4.38,...,2022,0.82,15,3,The time series has 3 missing value(s) within ...,to missing value(s),NaN,2008.0,2022.0,0
9,Albania,Kidnapping,visible,15,12,2008,2022,0.16,0.140,0.32,...,2011,0.08,15,3,The time series has 3 missing value(s) within ...,to missing value(s),NaN,2008.0,2022.0,0


In [7]:
# nejcasteji chybi v casovych radach par hodnot(udaje za 3, 1, 4 roky) v ramci delsiho vyplnovaciho obdobi
no_quality_data[['first_fill_year', 'last_fill_year', 'quality_range_unfill_data']].value_counts().reset_index().head() 

,first_fill_year,last_fill_year,quality_range_unfill_data,count
0,2008,2022,3,15
1,2008,2022,1,12
2,2008,2020,4,11
3,2019,2022,0,5
4,2008,2022,2,4


Pokud bychom chteli pracovat s temi kvalitneji vyplnenymi daty bez chybejich hodnot, mohli bychom postupovat takto.

In [8]:
# tzn. nechceme chybejici hodnoty v zaznamenavanem obdobi a chceme casovou radu alepson o 5 rocich vcetne. 
quality_data = df[(df['quality_range_unfill_data'] == 0) & (df['count_fill_values'] > 4)] #684 zaznamu
quality_data.head()

,country,crime,crime_category,count_years,count_fill_values,first_fill_year,last_fill_year,mean_value,median_value,max_value,...,min_value_year,standard_deviation,quality_range_fill_data,quality_range_unfill_data,missing_values_info,trend,relative_trend_strength,min_range_year,max_range_year,count_outliers
0,Albania,Acts against computer systems,hidden,15,7,2016,2022,3.89,2.93,7.14,...,2018,1.77,7,0,The time series has no missing values within t...,increasing,0.68,2008.0,2022.0,0
2,Albania,Bribery,hidden,15,7,2016,2022,10.29,9.55,17.68,...,2020,3.64,7,0,The time series has no missing values within t...,increasing,0.72,2008.0,2022.0,2
5,Albania,Child pornography,sensitive,15,6,2016,2021,0.88,0.17,4.38,...,2018,1.71,6,0,The time series has no missing values within t...,increasing,0.98,2008.0,2022.0,1
6,Albania,Corruption,hidden,15,7,2016,2022,37.86,35.99,46.46,...,2020,5.51,7,0,The time series has no missing values within t...,increasing,0.57,2008.0,2022.0,0
7,Albania,Fraud,hidden,15,7,2016,2022,32.25,32.55,35.39,...,2020,2.49,7,0,The time series has no missing values within t...,increasing,0.58,2008.0,2022.0,0


Summarizacni tabulka pro konkretni stat.

In [9]:
# Z par vybranych sloupcu muzeme jednoduse zjistit pro jednotlive zeme, jak jsou na tom v ramci kriminality. 
# jednoznacne je dulezita interpretace rustu/poklesu dle toho, o jakou kategorii tr. cinu se jedna!
trend_columns = ['country', 'crime', 'crime_category', 'count_years', 'quality_range_fill_data', 'quality_range_unfill_data', 'trend', 'relative_trend_strength', 'count_outliers']
switz_summ = df[df['country'] == 'Switzerland']
switz_summ_trend = switz_summ[trend_columns].sort_values(by='crime_category')
switz_summ_trend

,country,crime,crime_category,count_years,quality_range_fill_data,quality_range_unfill_data,trend,relative_trend_strength,count_outliers
819,Switzerland,Acts against computer systems,hidden,0,<NA>,<NA>,<NA>,NaN,0
830,Switzerland,Participation in an organized criminal group,hidden,15,7,0,decreasing,0.66,0
826,Switzerland,Fraud,hidden,15,7,0,increasing,0.82,0
825,Switzerland,Corruption,hidden,15,7,0,decreasing,0.99,0
829,Switzerland,Money laundering,hidden,15,7,0,increasing,1.00,0
821,Switzerland,Bribery,hidden,15,7,0,increasing,0.76,0
839,Switzerland,Unlawful acts involving controlled drugs or pr...,hidden,15,15,0,increasing,0.50,0
824,Switzerland,Child pornography,sensitive,15,2,0,decreasing,1.00,0
831,Switzerland,Rape,sensitive,15,15,0,increasing,0.64,1
834,Switzerland,Sexual assault,sensitive,15,14,0,decreasing,0.69,0


In [10]:
# muzeme hledat extremnejsi vykyvy v datatech, jednoduse si vyfiltrujeme data, kde máme spocitane outliers
# je potreba davat pozor na pripadnou interpretaci, protoze nektere tr. cinny mohou byt podkategorii jineho tr. cinu, viz readme

extreme_deviations_columns = ['country', 'crime', 'crime_category', 'count_years', 'max_value', 'max_value_year', 'min_value', 'min_value_year', 'standard_deviation', 'mean_value', 'count_outliers']

# z vysledku vidime, ze 2(Rape, Theft of a motorized vehicle or parts thereof) z 5 criminalnich cinu byly z podkategorii
switz_summ_extr = switz_summ[extreme_deviations_columns].sort_values(by='crime_category')
switz_summ_extr = switz_summ_extr[switz_summ_extr['count_outliers'] > 0]
switz_summ_extr


,country,crime,crime_category,count_years,max_value,max_value_year,min_value,min_value_year,standard_deviation,mean_value,count_outliers
831,Switzerland,Rape,sensitive,15,9.92,2022,6.46,2015,0.93,7.66,1
836,Switzerland,Sexual violence,sensitive,15,44.60,2017,32.00,2010,3.38,34.80,1
828,Switzerland,Kidnapping,visible,15,0.09,2018,0.00,2010,0.02,0.05,2
838,Switzerland,Theft of a motorized vehicle or parts thereof,visible,15,216.94,2022,72.63,2016,37.88,104.32,1
833,Switzerland,Serious assault,visible,15,117.11,2008,6.18,2011,28.38,14.54,2


In [11]:
# pro zemi si lze roztridit tr. ciny dle kategorie a trendu a silu trendu zprumerovat
# nejprve vsak musime odstranit 'duplicitni', respektive podkategorie jinych tr. cinu
switz_summ_no_subcategory = switz_summ[~switz_summ['crime'].isin(['Rape', 'Sexual assault', 'Child pornography', 'Burglary of private residential premises', 'Theft of a motorized vehicle or parts thereof', 'Bribery'])]

# na prvni pohled muzeme vycist, ze dochazi k castejsimu nahlasovat 'sensitive' tr.cinu a spise ubyva viditelnych tr. cinu
switz_summ_no_subcategory.groupby(['crime_category', 'trend'])['relative_trend_strength'].mean()

crime_category  trend     
hidden          decreasing    0.825000
                increasing    0.773333
sensitive       decreasing    0.530000
                increasing    0.930000
visible         decreasing    0.682000
                increasing    0.570000
Name: relative_trend_strength, dtype: float64

Summarizacni tabulka pro konkretni tr. cin.

In [12]:
# Z par vybranych sloupcu muzeme zjistit, jak jsou na tom trendove jednotlive krimi ciny a zda jsou v datech nejake podezrele udalosti
crime_summ = df[df['crime'] == 'Intentional homicide']
crime_summ = crime_summ[trend_columns].sort_values(by='country')
crime_summ.head(10)

,country,crime,crime_category,count_years,quality_range_fill_data,quality_range_unfill_data,trend,relative_trend_strength,count_outliers
8,Albania,Intentional homicide,visible,15,15,3,to missing value(s),NaN,0
29,Austria,Intentional homicide,visible,15,15,0,increasing,0.51,0
50,Belgium,Intentional homicide,visible,15,15,0,decreasing,0.59,0
71,Bosnia and Herzegovina,Intentional homicide,visible,15,5,0,decreasing,0.60,0
92,Bulgaria,Intentional homicide,visible,15,15,0,decreasing,0.69,0
113,Croatia,Intentional homicide,visible,15,15,0,decreasing,0.65,1
134,Cyprus,Intentional homicide,visible,15,15,0,decreasing,0.52,0
155,Czechia,Intentional homicide,visible,15,15,0,decreasing,0.58,0
176,Denmark,Intentional homicide,visible,15,15,0,increasing,0.51,1
197,England and Wales,Intentional homicide,visible,11,11,0,decreasing,0.52,0


In [13]:
# nasledna filtrace, ktera zeme ma nejvice rostouci a nejvice klesajici trend
strongest_increasing = crime_summ[crime_summ['trend'] == 'increasing'].sort_values(by='relative_trend_strength', ascending=False).head(1)
strongest_decreasing = crime_summ[crime_summ['trend'] == 'decreasing'].sort_values(by='relative_trend_strength', ascending=False).head(1)

print("Country with the strongest increasing trend:")
print(strongest_increasing[['country', 'crime', 'relative_trend_strength']])

print("\nCountry with the strongest decreasing trend:")
print(strongest_decreasing[['country', 'crime', 'relative_trend_strength']])

Country with the strongest increasing trend:
     country                 crime  relative_trend_strength
344  Iceland  Intentional homicide                      0.6

Country with the strongest decreasing trend:
       country                 crime  relative_trend_strength
470  Lithuania  Intentional homicide                     0.88


In [14]:
# lze si tr. cin roztridit dle trendu a vypocitat prumer sily daneho trendu
# v prumeru nam umyslnych vrazd spise ubyva

crime_summ.groupby('trend')['relative_trend_strength'].mean()

trend
decreasing             0.636552
increasing             0.536667
to missing value(s)         NaN
Name: relative_trend_strength, dtype: float64

Kdyz bychom chteli vytipovat zeme a tr. ciny, ktere stoji ta to blize prozkoumat. 

In [15]:
# tr. ciny zahrnujici podkategorie, na ktere se zamerit
suspicious_df = df[trend_columns]
suspicious_df = suspicious_df[suspicious_df['count_outliers'] > 0]
suspicious_df['crime'].value_counts()

crime
Bribery                                                   17
Sexual assault                                            16
Serious assault                                           14
Kidnapping                                                12
Theft                                                     12
Sexual violence                                           12
Rape                                                      11
Child pornography                                         10
Attempted intentional homicide                             9
Sexual exploitation                                        9
Corruption                                                 9
Acts against computer systems                              8
Money laundering                                           8
Fraud                                                      8
Theft of a motorized vehicle or parts thereof              6
Burglary                                                   6
Intentional homici

In [16]:
# zeme, na ktere se zamerit - obsahuje podkategorie
suspicious_df['country'].value_counts().reset_index().head()

,country,count
0,Germany,12
1,Estonia,10
2,Greece,9
3,Czechia,9
4,Spain,9


In [17]:
# zeme, na ktere se zamerit - neobsahuje podkategorie
suspicious_df_no_subcategory = suspicious_df[(suspicious_df['count_outliers'] > 0) & (~suspicious_df['crime'].isin(['Rape', 'Sexual assault', 'Child pornography', 'Burglary of private residential premises', 'Theft of a motorized vehicle or parts thereof', 'Bribery'])) ]
suspicious_df_no_subcategory['country'].value_counts().reset_index().head()

,country,count
0,Spain,8
1,Germany,8
2,Estonia,7
3,Greece,6
4,Croatia,6


Vrele doporucuji si vyzkouset dash app nebo interaktivni jupyter notebook. Na data nam poskytnou jiny uhel pohledu. 